In [73]:
import threading
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from translate import Translator #python翻譯套件
import googletrans #python google翻譯套件
from pprint import pprint
import random,time
from random import randint
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import jieba
import jieba.posseg as pseg
x=pd.read_csv('Recipe.csv',encoding='utf-8')
print(len(x['Ingredients']))
test=(x['Ingredients'])
#清除資料
def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)#去貼圖
    x = re.sub(r'[＜<《〈［(（【{][^>＞】)}）》〉］]*[>》＞】)}）〉］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
   
    x=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",x)
    x= re.sub(r'(一(包|碗|小碗|附)|(適|少)(量|當|許)|新鮮|市售|\
        |隔夜|切(段|絲|成|塊|片|碎末|花|碎|小丁|丁|末|細|條|小(塊|段))|約|(或|可(不用|用|依|選|均勻|不加|視家裡|\
        |替換|省略|切分|以)).*)|(皆|亦|即|也|均)可|(此次|一(個|支|大匙|小(撮|匙)|隻|把)|(依|視)個人|又(稱|名)).*|\
        |兩(?!節|層).*|數.*|少|個人包|成分|手切|去(皮|骨)|帶皮|現炒|原味|吃剩的|一(?!即|般)|小$.*|斜段|大碗|\
        |份量|沾醬|方便製作的|宜口為主|建議|表面裝飾|塑膠袋|喜歡|室溫|焦糖部分|焦糖材料|有機|.*刮刀|.*刀器|刀器|\
        |廚具刀|沾刀|刮刀|.*剪刀|美工刀|鋁罐|膠帶|砧板|玻璃密封罐|(or|OR)|的|加入|全聯|碗公|畫刀|料理機|食物保鮮袋|\
        |砧板|整粒|半顆|圓刀.*|抹刀|.*盤子|削皮器|溫度計|.*打蛋器|雕刻刀|美國棒球|.*蠟燭|.*水果刀|烤盤.*|.*烤模核金|.*拌盆|\
        |(大|中|小)塊|材料|中小型|冷藏|普通|退冰|完成|調味料|我用|是|蒸熟|不要太大|吋|金品|家樂福嚴選|(小|中|大)包裝|比較|\
        |煮開|(大|中|小)顆|超市可買到|(韓|日|美|台|泰)式|(日本|韓國|台灣|美國|泰國)|食材|子和風|碗|至要|http.*|北海道|罐裝|\
        |罐頭|玻璃罐|義式|炸粉|莫札|衛生|皮屑|過濾|開|帕瑪森|天使|調味|低脂|塗抹|羽衣|頭|清|黑豚|天使|食|牙籤|桔醬|虎牌|泡發\
        |家樂福|燙熟|工研|吃剩下|泡軟|條|純正|模型|黑白|香煎|冷凍|炒香|熱騰騰|陳年|至要|含皮拍裂|古法炭焙|苦甜|小號|融化|白色|\
        |自製流淚|即用|不锈鋼.*|圓形|義美|軟化後|雀巢水滴|無鋁|現榨|扣掉內餡|烘培用|內餡|防潮|大波露|耐高溫|無糖|法式|蜜汁|速發|\
        |打發|冷|元本山|自磨|七彩|自製|汆燙|地捫|中藥店|焗烤|燙|醃漬小|半包|炸|香辣|墨西哥|燒|薄片|好菇道|碎|.*脫水器|時令|西班牙|\
        |粗粒|調漿|紅黃色|幹洗淨|淡|包去粗梗|台東|梅取|牛頭牌|隔餐掉|與|粉白|澳洲|藍象|.*造型|片丁|煙燻|其他|史雲生|烹大師|半根|綜合|\
        |水煮|隔餐|冷掉的|掉|片盒|東石|香料共和國|去骨|.*廚具刀|鄉菇香黑|普格傳統|.*鉛筆機|.*量杯|.*餐夾|.*保鮮盒|印尼|.*調理機|急凍|\
        |初榨|烘焙用|加拿大|頂級|台糖|鄉菇香黑|細|黑早|.*濾網|粗黑|.*烤箱|乾燥|芝司樂|高鈣|安心|聖女|亨氏|.*棉巾|帶肥|可改做|Prego|\
        |福壽|濃縮液|特級|李錦記|農家瑞初榨|小支|大支|東寶|雲嶺鮮雞|利用|另一半|白綠|大成鹿野土雞|築地一番鮮|手剝|新食派|中型|手工|法國|\
        |樂事|快速|綠色|.*方形模|裝飾|高融點|冰|.*烤模|常溫|天然|過篩|光泉|天然雲尼拿|浸著|湯匙|任何口味|針|溶化|調色|脫殼|切粒|三花|.*桿麵棍|\
        |.*烤盤|.*創意餐廚|英國|.*刮板|光合農場|林復振|組合包|得意的一天|大創DIY|用的白醬|桂冠輕鬆生活|等等|顆|量|大匙|適量|龜甲萬料亭|\
        |決定|要不要|甜度|看|娘惹|即煮醬素|等|鹹度|濃稠度|調|克|用|輕|竹籤|即|平葉佳|口味|吃不慣|使用鹽|請以|好侍佛蒙特|屋|圓長型|高速公路|\
        |尼諾|煮至|彈牙|不建議|混雜|口味|以免|加上|本來|有|和|超市販售|之|煮好|放涼|經典|香濃白醬|禾法頌普羅旺斯|已|煮熟|並|壓成|泥狀|\
        |小磨坊|熱炒系列|統一|便利超商|另備些|外層|及|類|使用的|圖中|不加|我|要|做成|所以|正統派士|加|極品|有心|肉舖子|月球|太平洋品|\
        |養生|純醱酵|施巴護潔露.*|廣吉',"", x)
    x=re.sub("忌廉","鮮奶油",x)
    checkfood="火鍋"
    check_string=re.search(checkfood,x)
    if check_string!=None:
        #print("這是火鍋片類")
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)
        return x
    else:    
        x= re.sub(r'各$|各(式|種)|片狀|特級冷壓初榨|剝殼|方便製作的份量|不同|小(丁|束)|酌量|茄子重量的|沒餡|付煮熟|\
            |幾滴|裝飾用|煮切|去殼|醃料分成|鍋子.*|.*鍋|',"",x)
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)

        return x
#斷字斷詞中研院
def ckip_cut(cut_string):
    #GPU跑
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    ws = WS("./data", disable_cuda=False)#(WS) 斷詞
    pos = POS("./data", disable_cuda=False)# (POS) 詞性標注
    ner = NER("./data", disable_cuda=False)# (NER) 實體辨識
    word_to_weight = {"玫瑰鹽": 1,"牛背脊肉": 1,}
    dictionary = construct_dictionary(word_to_weight)
    #print(dictionary)
     
    sentence_list =cut_string#cut_string要用list包住

    word_sentence_list = ws(
        sentence_list,
      
    )
    word_sentence_list = ws(sentence_list, recommend_dictionary=dictionary)
    word_sentence_list = ws(sentence_list, coerce_dictionary=dictionary)
    pos_sentence_list = pos(word_sentence_list)

    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    del ws
    del pos
    del ner
    def print_word_pos_sentence(word_sentence, pos_sentence):
        save_word=[]
        assert len(word_sentence) == len(pos_sentence)

        for word, pos in zip(word_sentence, pos_sentence):
            #print(f"{word}({pos})", end="\u3000")
            a=word
            b=pos#顯示磁性
            #print("這個字{}的詞性是{}".format(a,b))

            if b =="Na":
                save_word.append(a)#存入暫存氣
        print(save_word)
        return save_word
     
    for i, sentence in enumerate(sentence_list):
        #print()
        #print(f"'{sentence}'")
        finsh=print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        return finsh
    
def jieba_cut2(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    cut_list=cut_word
    finsh=[]
    for str1 in cut_list:
        seg_list = jieba.lcut(str1, cut_all=False)
        #print(seg_list)
        for i in seg_list:
            finsh.append(i)
        
        #print(a)
        #print(finsh)
        
        #print(finsh)
    
    print(finsh)
    return finsh
    
def jieba_cut(cut_word):
    #jieba.set_dictionary('./dict.txt')
    finsh=[]
    text = cut_word
    #print(text)
    words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    for word, flag in words:
        if flag =="n" or flag== "nr" or flag=='x' or flag=='nz':
            print(word, flag)
            finsh.append(word)
        else:
            print(word, flag)
            #pass
    print(finsh)
    return finsh 

#英轉中    
def check_english_goole(string):
    checkstring=string 
    pattern='[a-zA-Z]{3,}' #英文3個字以上篩選
    pattern2='[\u4e00-\u9fa5]+'#選取中文
    #pattern2='[^[a-zA-Z]{3,}]'
    check_eng=re.search(pattern,checkstring)#判斷是否是英文單字
    check_tw=re.search(pattern2,checkstring)
    check_eng_finditer=list(re.finditer(pattern,checkstring))#用finditer找英文match值 回傳多筆資料
    check_tw_finditer=list(re.finditer(pattern2,checkstring))#用finditer找中文match值 回傳多筆資料
    check_zn_tw=re.findall(pattern2,checkstring)#中文字的範圍是個list
    #check_zn_tw= [i for i in check_zn_tw if i != '']#刪除空值
    condition=re.findall(pattern,checkstring)#英文字的範圍是個list

    set_order={}
    get_tw_order={}
    get_en_order={}
    
    for i in check_tw_finditer:
        get_order=i.start()#抓match初始位置
        get_items=i.group(0)#抓match的文字轉成str
         #print(get_items)
       
        tmp_dict={get_items:get_order}
        #print(tmp_dict)
        get_tw_order.update(tmp_dict)
        tmp_dict={}
   # print( get_tw_order)
    for j in check_eng_finditer:
        get_order=j.start()
        get_items=j.group(0)
        #print(get_items)
        #time.sleep(2)
        translator = googletrans.Translator()#用google套件
        change_eng=translator.translate( get_items, dest='zh-tw').text#英文轉中文
        #print(change_eng)
        #tmp_string="({},{})".format(  change_eng,get_order)
        tmp_dict={change_eng:get_order}
        #print(tmp_dict)
        get_en_order.update(tmp_dict)
        tmp_dict={}
    #print(get_en_order)
    #print(get_tw_order)
    #print(get_en_order)
    set_order.update(get_tw_order)
    set_order.update(get_en_order)
    #print(set_order)
    finsh_list=sorted(set_order.items(), key=lambda x:x[1])
    #print(finsh_list)
    finish=""
    for i in finsh_list:
        finish=finish+i[0]
    return  finish
    #重制所有dict
    set_order={}
    get_tw_order={}
    get_en_order={}
    finsh_list=[]
    
    
dict2={}
count=0
list_col=[]
for food_list in test:
    
    #print(type(food_list))
    try:
        food_list=food_list.replace("、"," ")
        food_list=food_list.replace(";"," ")
        #print(food_list)
        list3=food_list.split()
        re_a_count=1
        #print(list3)
        #print(type(list2))
        #count+=1
        #count=count+1
        #print("這是第{}個欄位".format(count))
        #print("__________________________")
        save_back=''
        save_back_list=[]
        #print(count)

        #print("第{}個欄位".format(count))
        count+=1
        for a in list3:
            #print(a)
            #print(a)
            re_a=preprocess(a)

            re_a=check_english_goole(re_a)
            #re_a=preprocess(a)
            #re_a=jieba_cut(re_a)
            

            if len(re_a)>10:
                print(a)
                print("斷詞前",re_a)
                #print("字數大於0個進入斷字斷詞")
                print("第{}個欄位".format(count))
                re_a=[re_a]
                #re_a_list=ckip_cut(re_a)
                #re_a_list=jieba_cut(re_a)
                re_a_list=jieba_cut2(re_a)
                #print(re_a_list)

                for i in re_a_list:
                    re_a=i

                    save_back=save_back+re_a+" "

                    if re_a in dict2:

                        dict2[re_a]+=1

                    else:
                        dict2[re_a] =1
                        #print(re_a_count)
                    re_a_count+=1
                    #print(i,re_a)
                    #print(save_back)
                    #print("_______")
            else:

                save_back=save_back+re_a+" "

                #print(save_back)
                #print("+========+")
                #print(re_a)

                if re_a in dict2:

                    dict2[re_a]+=1

                else:
                    dict2[re_a] =1
                #print(re_a_count)
                re_a_count+=1


        #save_back=list(save_back)
        save_back_list=[save_back]
        list_col.append(save_back_list)
        save_back=''
        save_back_list=[]
    except AttributeError as e:
        continue
#print(dict2)
new_list1=sorted(dict2.items(),key=lambda item:item[1])
#print(new_list)
get_dict1=dict(new_list1)
#print(get_dict1)
#print(len(get_dict1))
#print(list_col)
',"", x)
    x=re.sub("忌廉","鮮奶油",x)
    checkfood="火鍋"
    check_string=re.search(checkfood,x)
    if check_string!=None:
        #print("這是火鍋片類")
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)
        return x
    else:    
        x= re.sub(r'各$|各(式|種)|片狀|特級冷壓初榨|剝殼|方便製作的份量|不同|小(丁|束)|酌量|茄子重量的|沒餡|付煮熟|\
            |幾滴|裝飾用|煮切|去殼|醃料分成|鍋子.*|.*鍋|',"",x)
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)

        return x
#斷字斷詞中研院
def ckip_cut(cut_string):
    #GPU跑
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    ws = WS("./data", disable_cuda=False)#(WS) 斷詞
    pos = POS("./data", disable_cuda=False)# (POS) 詞性標注
    ner = NER("./data", disable_cuda=False)# (NER) 實體辨識
    word_to_weight = {"玫瑰鹽": 1,"牛背脊肉": 1,}
    dictionary = construct_dictionary(word_to_weight)
    #print(dictionary)
     
    sentence_list =cut_string#cut_string要用list包住

    word_sentence_list = ws(
        sentence_list,
      
    )
    word_sentence_list = ws(sentence_list, recommend_dictionary=dictionary)
    word_sentence_list = ws(sentence_list, coerce_dictionary=dictionary)
    pos_sentence_list = pos(word_sentence_list)

    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    del ws
    del pos
    del ner
    def print_word_pos_sentence(word_sentence, pos_sentence):
        save_word=[]
        assert len(word_sentence) == len(pos_sentence)

        for word, pos in zip(word_sentence, pos_sentence):
            #print(f"{word}({pos})", end="\u3000")
            a=word
            b=pos#顯示磁性
            #print("這個字{}的詞性是{}".format(a,b))

            if b =="Na":
                save_word.append(a)#存入暫存氣
        print(save_word)
        return save_word
     
    for i, sentence in enumerate(sentence_list):
        #print()
        #print(f"'{sentence}'")
        finsh=print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        return finsh
    
def jieba_cut2(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    cut_list=cut_word
    finsh=[]
    for str1 in cut_list:
        seg_list = jieba.lcut(str1, cut_all=False)
        #print(seg_list)
        for i in seg_list:
            finsh.append(i)
        
        #print(a)
        #print(finsh)
        
        #print(finsh)
    
    print(finsh)
    return finsh
    
def jieba_cut(cut_word):
    #jieba.set_dictionary('./dict.txt')
    finsh=[]
    text = cut_word
    #print(text)
    words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    for word, flag in words:
        if flag =="n" or flag== "nr" or flag=='x' or flag=='nz':
            print(word, flag)
            finsh.append(word)
        else:
            print(word, flag)
            #pass
    print(finsh)
    return finsh 

#英轉中    
def check_english_goole(string):
    checkstring=string 
    pattern='[a-zA-Z]{3,}' #英文3個字以上篩選
    pattern2='[\u4e00-\u9fa5]+'#選取中文
    #pattern2='[^[a-zA-Z]{3,}]'
    check_eng=re.search(pattern,checkstring)#判斷是否是英文單字
    check_tw=re.search(pattern2,checkstring)
    check_eng_finditer=list(re.finditer(pattern,checkstring))#用finditer找英文match值 回傳多筆資料
    check_tw_finditer=list(re.finditer(pattern2,checkstring))#用finditer找中文match值 回傳多筆資料
    check_zn_tw=re.findall(pattern2,checkstring)#中文字的範圍是個list
    #check_zn_tw= [i for i in check_zn_tw if i != '']#刪除空值
    condition=re.findall(pattern,checkstring)#英文字的範圍是個list

    set_order={}
    get_tw_order={}
    get_en_order={}
    
    for i in check_tw_finditer:
        get_order=i.start()#抓match初始位置
        get_items=i.group(0)#抓match的文字轉成str
         #print(get_items)
       
        tmp_dict={get_items:get_order}
        #print(tmp_dict)
        get_tw_order.update(tmp_dict)
        tmp_dict={}
   # print( get_tw_order)
    for j in check_eng_finditer:
        get_order=j.start()
        get_items=j.group(0)
        #print(get_items)
        #time.sleep(2)
        translator = googletrans.Translator()#用google套件
        change_eng=translator.translate( get_items, dest='zh-tw').text#英文轉中文
        #print(change_eng)
        #tmp_string="({},{})".format(  change_eng,get_order)
        tmp_dict={change_eng:get_order}
        #print(tmp_dict)
        get_en_order.update(tmp_dict)
        tmp_dict={}
    #print(get_en_order)
    #print(get_tw_order)
    #print(get_en_order)
    set_order.update(get_tw_order)
    set_order.update(get_en_order)
    #print(set_order)
    finsh_list=sorted(set_order.items(), key=lambda x:x[1])
    #print(finsh_list)
    finish=""
    for i in finsh_list:
        finish=finish+i[0]
    return  finish
    #重制所有dict
    set_order={}
    get_tw_order={}
    get_en_order={}
    finsh_list=[]
    
    
dict2={}
count=0
list_col=[]
for food_list in test:
    
    #print(type(food_list))
    try:
        food_list=food_list.replace("、"," ")
        food_list=food_list.replace(";"," ")
        #print(food_list)
        list3=food_list.split()
        re_a_count=1
        #print(list3)
        #print(type(list2))
        #count+=1
        #count=count+1
        #print("這是第{}個欄位".format(count))
        #print("__________________________")
        save_back=''
        save_back_list=[]
        #print(count)

        #print("第{}個欄位".format(count))
        count+=1
        for a in list3:
            #print(a)
            #print(a)
            re_a=preprocess(a)

            re_a=check_english_goole(re_a)
            #re_a=preprocess(a)
            #re_a=jieba_cut(re_a)
            

            if len(re_a)>10:
                print(a)
                print("斷詞前",re_a)
                #print("字數大於0個進入斷字斷詞")
                print("第{}個欄位".format(count))
                re_a=[re_a]
                #re_a_list=ckip_cut(re_a)
                #re_a_list=jieba_cut(re_a)
                re_a_list=jieba_cut2(re_a)
                #print(re_a_list)

                for i in re_a_list:
                    re_a=i

                    save_back=save_back+re_a+" "

                    if re_a in dict2:

                        dict2[re_a]+=1

                    else:
                        dict2[re_a] =1
                        #print(re_a_count)
                    re_a_count+=1
                    #print(i,re_a)
                    #print(save_back)
                    #print("_______")
            else:

                save_back=save_back+re_a+" "

                #print(save_back)
                #print("+========+")
                #print(re_a)

                if re_a in dict2:

                    dict2[re_a]+=1

                else:
                    dict2[re_a] =1
                #print(re_a_count)
                re_a_count+=1


        #save_back=list(save_back)
        save_back_list=[save_back]
        list_col.append(save_back_list)
        save_back=''
        save_back_list=[]
    except AttributeError as e:
        continue
#print(dict2)
new_list1=sorted(dict2.items(),key=lambda item:item[1])
#print(new_list)
get_dict1=dict(new_list1)
#print(get_dict1)
#print(len(get_dict1))
#print(list_col)
#9439


C:\Users\Big data\anaconda3\envs\test37\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


108724
紅椒粉,辣椒粉,洋蔥粉,白胡椒粉,蒜瓣,鹽
斷詞前 紅椒粉辣椒粉洋蔥胡椒粉蒜瓣鹽
第286個欄位
['紅椒粉', '辣椒粉', '洋蔥', '胡椒粉', '蒜瓣', '鹽']
ELECTBBQ烤雞翅
斷詞前 ELECTBBQ烤雞翅
第1170個欄位
['ELECTBBQ', '烤雞翅']
紅蘿蔔馬鈴薯木耳洋蔥紅椒
斷詞前 紅蘿蔔馬鈴薯木耳洋蔥紅椒
第2082個欄位
['紅蘿蔔', '馬', '鈴薯', '木耳', '洋蔥', '紅椒']
鮮奶油CrèmeFraîche
斷詞前 鮮奶油CrmeFrache
第2216個欄位
['鮮奶油', 'CrmeFrache']
Caciocavallo乳酪
斷詞前 Caciocavallo乳酪
第2353個欄位
['Caciocavallo', '乳酪']
Parmigiano
斷詞前 帕馬森（Parmigiano）
第2379個欄位
['帕馬森', '（', 'Parmigiano', '）']
Tartiflette
斷詞前 Tartiflette
第2480個欄位
['Tartiflette']
歐芹（即巴西利，平葉佳，捲葉可
斷詞前 歐芹即巴西利平葉佳捲葉可
第3307個欄位
['歐芹', '即', '巴西利', '平葉佳', '捲', '葉', '可']
大蒜末2瓣檸檬汁一顆量羅望子醬一匙魚露2大匙椰糖適量
斷詞前 大蒜末瓣檸檬汁羅望子醬匙魚露椰糖
第3670個欄位
['大蒜', '末瓣', '檸檬汁羅望子', '醬匙', '魚露', '椰糖']
米酒.胡椒粉.五香粉.油蔥酥
斷詞前 米酒胡椒粉五香粉油蔥酥
第4466個欄位
['米酒', '胡椒粉', '五香粉', '油蔥酥']
花椒乾peppercorn
斷詞前 花椒乾Peppercn
第4574個欄位
['花椒', '乾', 'Peppercn']
「香料2」八角，桂皮，花椒，百果，甘草
斷詞前 香料八角桂皮花椒百果甘草
第4793個欄位
['香料', '八角', '桂皮', '花椒', '百果', '甘草']
綜合魚肉片鯛魚片,鱈魚等白肉魚
斷詞前 魚肉片鯛魚片鱈魚白肉魚
第5544個欄位
['魚肉片', '鯛魚片', '鱈魚', '白肉魚']
咖哩塊，馬鈴薯塊，紅蘿蔔塊
斷詞前 咖哩塊馬鈴薯塊紅蘿蔔塊
第5748個欄位
['咖哩塊', '馬鈴薯塊

廣吉糙米片紫米香芋口味粉
斷詞前 廣吉糙米片紫米香芋口味粉
第20442個欄位
['廣吉', '糙米', '片紫米', '香芋', '口味', '粉']
黑胡椒/蕃茄醬/千島沙拉醬/蜂蜜芥末醬
斷詞前 黑胡椒蕃茄醬千島沙拉醬蜂蜜芥末醬
第20458個欄位
['黑胡椒', '蕃茄', '醬', '千島沙拉醬', '蜂蜜', '芥末', '醬']
蒜,洋蔥,彩椒,紅蘿蔔,蕃茄,洋茹,火腿
斷詞前 蒜洋蔥彩椒紅蘿蔔蕃茄洋茹火腿
第20716個欄位
['蒜', '洋蔥', '彩椒', '紅蘿蔔', '蕃茄', '洋茹', '火腿']
▊肉片調味▊米酒．白醋．水．醬油（或蠔油）．香油．太白炒與沙拉油
斷詞前 肉片米酒白醋水醬油香油太白炒沙拉油
第20810個欄位
['肉片', '米酒', '白醋', '水醬', '油', '香油', '太白', '炒', '沙拉油']
米鬆餅粉/蓬萊米粉/低筋麵粉
斷詞前 米鬆餅粉蓬萊米粉低筋麵粉
第20933個欄位
['米', '鬆', '餅', '粉', '蓬', '萊', '米粉', '低筋', '麵粉']
鋁箔紙剪成比瓶口大的正方形
斷詞前 鋁箔紙剪成比瓶口大正方形
第22575個欄位
['鋁箔紙', '剪成', '比', '瓶口', '大', '正方形']
RDF普羅旺斯無花果果醬
斷詞前 RDF普羅旺斯無花果果醬
第23179個欄位
['RDF', '普羅旺斯', '無', '花果', '果醬']
天然風味料理素-紫薯甜菜甘味
斷詞前 風味料理素紫薯甜菜甘味
第23202個欄位
['風味', '料理', '素紫薯', '甜菜', '甘味']
美生菜,苜蓿芽,紅椒,花椰菜
斷詞前 美生菜苜蓿芽紅椒花椰菜
第23206個欄位
['美', '生菜', '苜蓿', '芽紅椒', '花椰菜']
蘋果,芭樂,小黃瓜,藍莓,鳳梨
斷詞前 蘋果芭樂小黃瓜藍莓鳳梨
第23206個欄位
['蘋果', '芭樂', '小黃瓜', '藍莓', '鳳梨']
日田天領水。福岡極品八女茶。
斷詞前 日田天領水福岡極品八女茶
第23303個欄位
['日田天', '領水福岡', '極品', '八女', '茶']
b有鹽奶油（可無鹽，鹹味增層次
斷詞前 有鹽奶油可無鹽鹹味增層次
第23533個欄位
['有', '鹽', '奶油', '可無鹽', '鹹', '味', '

黑胡椒粉/匈牙利紅椒粉/鹽
斷詞前 黑胡椒粉匈牙利紅椒粉鹽
第43902個欄位
['黑', '胡椒粉', '匈牙利', '紅椒粉', '鹽']
香腸世家紅酒飛魚卵香腸
斷詞前 香腸世家紅酒飛魚卵香腸
第44223個欄位
['香腸', '世家', '紅酒', '飛魚', '卵', '香腸']
荳蔲2g，亦可至中藥行直接買滷包〕
斷詞前 荳蔲至中藥行直接買滷包
第44865個欄位
['荳', '蔲', '至', '中', '藥', '行', '直接', '買', '滷', '包']
Panasonic製麵包機SD-BMS105T
斷詞前 松下製麵包機SDBMST
第44916個欄位
['松下', '製', '麵', '包', '機', 'SDBMST']
Panasonic製麵包機SD-BMS105T
斷詞前 松下製麵包機SDBMST
第44919個欄位
['松下', '製', '麵', '包', '機', 'SDBMST']
c.蜂蜜+檸檬汁+胡椒+少許鹽+橄欖油
斷詞前 蜂蜜檸檬汁胡椒鹽橄欖油
第45110個欄位
['蜂蜜', '檸檬汁', '胡椒', '鹽', '橄欖油']
新北健康三寶-三之57號黃金甘藷
斷詞前 新北健康三寶三之號黃金甘藷
第45194個欄位
['新北', '健康', '三寶三之號', '黃', '金', '甘藷']
MoMo購物台-韓國Pure-Nature天然環保萬用料理紙
斷詞前 MOMO購物台純自然環保萬料理紙
第45197個欄位
['MOMO', '購物', '台純', '自然', '環保萬', '料理', '紙']
丸之家私房煲膠原經典高湯
斷詞前 丸之家私房煲膠原經典高湯
第45218個欄位
['丸', '之家', '私房', '煲', '膠原', '經典', '高湯']
五香粉蒜片太白粉香油糖
斷詞前 五香粉蒜片太白粉香油糖
第45401個欄位
['五香粉', '蒜片', '太白粉', '香油', '糖']
味全健康廚房黃金玄米油
斷詞前 味全健康廚房黃金玄米油
第45501個欄位
['味全', '健康', '廚房', '黃', '金玄米', '油']
配菜~燙熟玉米，花椰菜，紅蘿蔔，鑫鑫腸，魚板，小蕃茄
斷詞前 配菜玉米花椰菜紅蘿蔔鑫鑫腸魚板小蕃茄
第45521個欄位
['配菜', '玉米', '花椰菜', '紅蘿蔔', '鑫

配料:木耳.豆皮.豆腐.菇類.金針.丸子
斷詞前 配料木耳豆皮豆腐菇類金丸子
第64013個欄位
['配料', '木耳', '豆皮', '豆腐', '菇', '類金', '丸子']
France諾曼第卡門貝爾乾酪
斷詞前 法國諾曼第卡門貝爾乾酪
第64023個欄位
['法國諾曼', '第卡門', '貝爾', '乾酪']
新鮮橙汁150cc香魁克15cc丁香10顆無鹽奶油20克檸檬汁一小匙二人份
斷詞前 橙汁香魁丁香無鹽奶油檸檬汁
第64065個欄位
['橙汁', '香魁', '丁香', '無鹽奶油', '檸檬汁']
乾粉絲，水中浸後在沸水煮至軟身
斷詞前 乾粉絲水中浸後在沸至軟身
第64181個欄位
['乾粉', '絲', '水中', '浸後在', '沸至', '軟身']
特製醬汁柴魚or昆布醬油皆可傳統醬油地瓜粉水
斷詞前 特製醬汁柴魚昆布醬油傳統醬油地瓜粉水
第64831個欄位
['特', '製', '醬汁', '柴魚', '昆布', '醬油', '傳統', '醬油', '地瓜粉', '水']
花椰菜／紅蘿蔔／黃椒紅椒／小玉米
斷詞前 花椰菜紅蘿蔔黃椒紅椒小玉米
第64840個欄位
['花椰菜', '紅蘿蔔', '黃', '椒', '紅椒', '小', '玉米']
綜合海鮮料(鮮花枝片,鮮蝦仁,章魚...
斷詞前 海鮮料鮮花枝片鮮蝦仁章魚
第64850個欄位
['海鮮料', '鮮', '花枝', '片鮮', '蝦仁', '章魚']
各種菇類：我用的是金針菇和泡過的乾香菇，日式茸菇醬裡的基本款是金針菇，其他冰箱裡有的菇類也可以切一切一起加入。
斷詞前 菇類金菇和泡過乾香菇茸菇醬裡基本款金菇箱裡有菇類以切切起
第64857個欄位
['菇', '類金菇', '和', '泡過', '乾', '香菇', '茸菇', '醬裡', '基本', '款金菇', '箱裡', '有菇', '類以', '切切', '起']
培根,高麗菜,紅蘿蔔絲,玉米,蔥,蛋
斷詞前 培根高麗菜紅蘿蔔絲玉米蔥蛋
第64890個欄位
['培根', '高麗菜', '紅蘿蔔', '絲', '玉米', '蔥蛋']
中筋麵粉,鰹魚高湯,塩,日本山藥泥
斷詞前 中筋麵粉鰹魚高湯塩山藥泥
第64890個欄位
['中筋麵粉', '鰹魚', '高湯', '塩', '山藥', '泥']
蒜頭-辣椒--香菜--米酒--鹽巴

...

雞柳1盒；青瓜2條蒜頭4瓣；嫩薑1/4條蔥2根
斷詞前 雞柳盒青瓜蒜瓣嫩薑蔥根
第79371個欄位
['雞柳盒', '青瓜', '蒜瓣', '嫩', '薑', '蔥', '根']
醬油2大匙；白醋1大匙糖1小匙；麻油1小匙胡椒鹽2小匙；鹽1小匙太白粉1小匙；花
斷詞前 醬油白醋糖小匙麻油小匙胡椒鹽小匙鹽小匙太白粉小匙花
第79371個欄位
['醬油', '白醋', '糖', '小匙', '麻油', '小匙', '胡椒', '鹽', '小匙', '鹽', '小匙', '太白粉', '小匙', '花']
玉桂葉3塊原粒黑胡椒2湯匙黑啤1罐
斷詞前 玉桂葉塊原粒黑胡椒黑啤罐
第79533個欄位
['玉桂葉塊', '原粒', '黑胡椒', '黑啤', '罐']
辣椒粉1/4小匙,食油1湯匙,胡椒粉1/8小匙
斷詞前 辣椒粉小匙油胡椒粉小匙
第79634個欄位
['辣椒粉', '小匙', '油', '胡椒粉', '小匙']
鹿窯菇事-有機黑面白背乾木耳
斷詞前 鹿窯菇事黑面白背乾木耳
第79642個欄位
['鹿窯菇', '事', '黑面', '白背', '乾', '木耳']
鹿窯菇事-料理菇事-薑汁豬肉粥
斷詞前 鹿窯菇事料理菇事薑汁豬肉粥
第79643個欄位
['鹿窯菇', '事', '料理', '菇', '事', '薑', '汁', '豬肉', '粥']
鹿窯菇事-料理菇事系列-桂圓木耳粥
斷詞前 鹿窯菇事料理菇事系列桂圓木耳粥
第79645個欄位
['鹿窯菇', '事', '料理', '菇', '事', '系列', '桂圓', '木耳', '粥']
鹿窯菇事-料理菇事系列-起士蕈姑粥
斷詞前 鹿窯菇事料理菇事系列起士蕈姑粥
第79646個欄位
['鹿窯菇', '事', '料理', '菇', '事', '系列', '起士', '蕈', '姑粥']
鹿窯菇事-料理菇事系列-薑汁豬肉粥
斷詞前 鹿窯菇事料理菇事系列薑汁豬肉粥
第79648個欄位
['鹿窯菇', '事', '料理', '菇', '事', '系列', '薑', '汁', '豬肉', '粥']
鹿窯菇事-料理菇事系列-蒜香雞茸粥
斷詞前 鹿窯菇事料理菇事系列蒜香雞茸粥
第79651個欄位
['鹿窯菇', '事', '料理', '菇', '事', '系列', '蒜', '香雞', '茸', '粥']
鹿窯菇事-料理菇

苦甜巧克力塊140克無鹽牛油110克室溫中雞蛋3隻
斷詞前 巧力塊無鹽牛油中雞蛋隻
第83317個欄位
['巧力', '塊', '無鹽牛油', '中雞', '蛋', '隻']
忌廉芝士（Creamcheese)300克酸忌廉140克白糖70克雞蛋2隻消化餅
斷詞前 鮮奶油芝士酸鮮奶油白糖雞蛋隻消化餅
第83321個欄位
['鮮奶油', '芝士', '酸', '鮮奶油', '白糖', '雞蛋', '隻', '消化', '餅']
三文魚排2塊乾羅勒driedbasil1茶匙（teaspoon）紅椒粉spanish
斷詞前 三文魚排塊乾羅勒幹羅勒茶匙紅椒粉西班牙文
第83322個欄位
['三文', '魚排', '塊', '乾', '羅勒', '幹', '羅勒', '茶匙', '紅椒粉', '西班牙文']
黑胡椒碎blackpepper0.5茶匙（teaspoon）蜜糖芥茉honeymusta
斷詞前 黑胡椒茶匙蜜糖芥茉蜜mus
第83322個欄位
['黑胡椒', '茶匙', '蜜糖', '芥茉', '蜜', 'mus']
冬菇，雲耳，韓式粉絲，蝦米，紅蘿蔔絲，紹菜絛，豆腐粒，蛋絲
斷詞前 冬菇雲耳粉絲蝦米紅蘿蔔絲紹菜絛豆腐粒蛋絲
第83333個欄位
['冬菇', '雲耳粉', '絲', '蝦', '米', '紅蘿蔔', '絲紹', '菜', '絛', '豆腐', '粒蛋絲']
調味料蠔油2湯匙生抽2茶匙老抽2茶匙糖2-3茶匙黑胡椒碎2湯匙
斷詞前 蠔油生抽茶匙老抽茶匙糖茶匙黑胡椒
第83381個欄位
['蠔油', '生抽', '茶匙', '老抽', '茶匙', '糖', '茶匙', '黑胡椒']
指天椒粒/辣椒乾適量辣醬3茶匙(冠益/是拉差)蠔油2茶匙老抽1湯匙魚露
斷詞前 指天椒粒辣椒乾辣醬茶匙蠔油茶匙老抽魚露
第83383個欄位
['指天椒粒', '辣椒', '乾辣醬', '茶匙', '蠔油', '茶匙', '老抽', '魚露']
雲吞皮30張蔥花適量豬肉碎300克黑木耳末適量
斷詞前 雲吞皮張蔥花豬肉黑木耳末
第83384個欄位
['雲吞皮', '張蔥花', '豬肉', '黑木耳', '末']
醬汁甜豉油1湯匙蠔油1湯匙蔴油1湯匙鎮江醋1湯匙
斷詞前 醬汁甜豉油蠔油蔴油鎮江醋
第83384個欄位
['醬汁', '甜豉', '油', '蠔油', '蔴', '油', '

whippingcream液態忌廉/攪打奶油
斷詞前 淡奶油液態鮮奶油攪打奶油
第93828個欄位
['淡', '奶油', '液態', '鮮奶油', '攪', '打', '奶油']
車打芝士絲及馬蘇里拉芝士絲
斷詞前 車打芝士絲及馬蘇里拉芝士絲
第93840個欄位
['車打', '芝士', '絲及', '馬', '蘇', '里拉', '芝士', '絲']
BettyCrockerYellowCakeMix（432克）
斷詞前 貝蒂·克羅克（BettyCrocker）黃色蛋糕混合
第93964個欄位
['貝蒂', '·', '克羅克', '（', 'BettyCrocker', '）', '黃', '色', '蛋糕', '混合']
KefirStarter
斷詞前 KefirStarter
第93972個欄位
['KefirStarter']
其他如迷迭香葉,百里香,九層塔等香料
斷詞前 如迷迭香葉百里香九層塔香料
第94039個欄位
['如', '迷迭香', '葉', '百里香', '九層塔', '香料']
Parmesan芝士粉/低納硬芝士碎(原食譜35克)
斷詞前 巴馬干酪芝士粉低納硬芝士
第94245個欄位
['巴馬', '干酪', '芝士', '粉低', '納硬', '芝士']
藥材湯包，蜜棗，雲芝，白芍，茶樹菇，沙參，姬松茸，女貞子，杞子
斷詞前 藥材湯包蜜棗雲芝白芍茶樹菇沙參姬松茸女貞子杞子
第94350個欄位
['藥材湯', '包蜜', '棗雲芝', '白芍', '茶樹菇', '沙', '參姬', '松茸', '女貞子', '杞子']
老干媽香辣脆油辣椒醬/其他辣椒醬
斷詞前 老干媽脆油辣椒醬辣椒醬
第94400個欄位
['老干媽', '脆油', '辣椒', '醬', '辣椒', '醬']
披薩麵糰（PizzaDough)食材：3-3/4杯中筋麵粉。1-1/2杯温水。1湯匙糖
斷詞前 披薩麵糰杯中筋麵粉杯温水糖
第94526個欄位
['披薩', '麵', '糰', '杯', '中筋麵粉', '杯', '温水', '糖']
FoodWise的品質凍肉--澳洲MacroMeatsParoo袋鼠尾
斷詞前 FoodWise品質凍肉大肉類袋鼠尾
第94600個欄位
['FoodWise', '品質', '凍肉', '大肉', '類', '袋鼠', '尾']
食

水(根據個人喜歡傑粥或稀粥可加可減)我的經驗一般是小米和水的比例爲1:16.800ml
斷詞前 水我經驗一般小米和水比例爲
第100233個欄位
['水', '我', '經驗', '一般', '小米', '和', '水', '比例', '爲']
炒熟的糯米粉cookedGlutinousRiceFlour
斷詞前 炒熟糯米粉煮熟的糯米粉
第100349個欄位
['炒熟', '糯米粉', '煮熟', '的', '糯米粉']
纯牛奶如果紫薯比较水份多可適當減少20g
斷詞前 纯牛奶如果紫薯比较水份多可減
第100360個欄位
['纯牛奶', '如果', '紫薯', '比较', '水份', '多可減']
動物性鮮奶油(heavywhippingcre
斷詞前 動物性鮮奶油重鞭cre
第100442個欄位
['動物性鮮奶油', '重鞭', 'cre']
培根，義大利風乾培根更好
斷詞前 培根義大利風乾培根更好
第100442個欄位
['培根', '義', '大利', '風乾', '培根', '更好']
鹽之花FleurdeSel（
斷詞前 鹽之花FleurdeSel
第100493個欄位
['鹽之花', 'FleurdeSel']
小麥蛋白粉VitalWheatGluten（可省略或以高筋麵粉取代）
斷詞前 小麥蛋白粉重要小麥麵筋
第100505個欄位
['小麥', '蛋白粉', '重要', '小麥', '麵', '筋']
黃金馬鈴薯YukonGoldPotatoes
斷詞前 黃金馬鈴薯Yukon gold potatoes
第100514個欄位
['黃', '金', '馬鈴薯', 'Yukon', ' ', 'gold', ' ', 'potatoes']
愛爾蘭切達起司，刨絲IrishCheddarCheese
斷詞前 愛爾蘭切達起司刨絲切達干酪奶酪
第100514個欄位
['愛爾蘭切', '達', '起司', '刨', '絲切', '達', '干酪', '奶酪']
未精製粗糖turbinadosugar
斷詞前 未精製粗糖Turbinado糖
第100516個欄位
['未', '精', '製', '粗糖', 'Turbinado', '糖']
全脂瑞可達起司whole-milkricottacheese
斷詞前 全脂瑞可達起司全脂牛奶
第100517個欄位
['全脂'

椰奶(椰漿)74ml,淡忌廉80ml,砂糖32g,無鹽牛油50g.
斷詞前 椰奶鮮奶油砂糖無鹽牛油
第105581個欄位
['椰奶', '鮮奶油', '砂糖', '無鹽牛油']
Parmaham意大利巴馬焙根
斷詞前 帕爾馬火腿意大利巴馬焙根
第105590個欄位
['帕爾馬', '火腿', '意大利', '巴馬', '焙根']
請參考法式國王派FrenchKings'Cake的派皮作法→http://cook1coo
斷詞前 請參考國王派法國國王蛋糕派皮作法
第105958個欄位
['請', '參考國', '王派', '法國國王', '蛋糕', '派皮', '作法']
雞蛋，發打並煎成蛋餅，待涼後切成小塊備用
斷詞前 雞蛋發打並煎成蛋餅待涼後備
第106034個欄位
['雞蛋發', '打並', '煎成', '蛋餅', '待', '涼後備']
桑寄生，隨手一札，入湯袋，沖洗乾淨
斷詞前 桑寄生隨手札入湯袋沖洗乾淨
第106077個欄位
['桑寄生', '隨', '手札', '入', '湯袋', '沖', '洗', '乾淨']
忌廉芝士奶油乳酪CreamCheese
斷詞前 鮮奶油芝士奶油乳酪奶油乳酪
第106209個欄位
['鮮奶油', '芝士', '奶油', '乳酪', '奶油', '乳酪']
玉米澱粉cornstarch
斷詞前 玉米澱粉cnstarch
第106209個欄位
['玉米', '澱粉', 'cnstarch']
玉米粉cornstarch
斷詞前 玉米粉cnstarch
第106221個欄位
['玉米粉', 'cnstarch']
美生菜,苜蓿芽,紅椒,花椰菜
斷詞前 美生菜苜蓿芽紅椒花椰菜
第106362個欄位
['美', '生菜', '苜蓿', '芽紅椒', '花椰菜']
蘋果,芭樂,小黃瓜,藍莓,鳳梨
斷詞前 蘋果芭樂小黃瓜藍莓鳳梨
第106362個欄位
['蘋果', '芭樂', '小黃瓜', '藍莓', '鳳梨']
醬油,鹽,烏醋,五香粉,胡椒粉
斷詞前 醬油鹽烏醋五香粉胡椒粉
第106394個欄位
['醬油', '鹽', '烏醋', '五香粉', '胡椒粉']
香腸世家-泡菜飛魚卵香腸
斷詞前 香腸世家泡菜飛魚卵香腸
第106410個欄位
['香腸', '世家', '泡菜', '飛魚', '卵', '香腸']
施巴護潔露黃金女郎加強型
斷

In [72]:



import pandas as ps
data=ps.DataFrame(list_col,columns=['Ingredients_clean'])
print(data)
data.to_csv('database_back_jieba_cut_5.csv',header=True,index=False,encoding='utf-8-sig')

                                        Ingredients_clean
0                              五花肉 蛋 薯粉 南乳紅腐乳 鹽巴 五香粉 胡椒粉 
1       雞胸肉 醬油膏 蜂蜜 鹽 黑胡椒 薑末 蒜末 辣椒醬 醬油 蜂蜜 醋 二砂糖 白芝麻 水 玉...
2                            馬鈴薯 蒜 奶油 水 牛奶 鹽 香料 白胡椒 瑞拉起司 
3                                 義大利肉醬 乳酪絲 花椰菜 馬鈴薯 牛奶 鹽 
4                      馬鈴薯 培根 洋蔥 黑胡椒粉 鹽 鮮奶油 乳酪絲 奶油 義大利香料 
...                                                   ...
108719                                         米酒青蔥 蝦子薑片 
108720                          小卷青蔥九層塔 蒜辣椒 黑麻油米酒魚露醬油 薑片 
108721                         在養生黑麻油 黃耆枸杞薑 青江菜米酒塩巴 羊肉當歸 
108722                           烏骨雞 蒜米酒塩巴 蛤蜊九層塔 在養生小磨香油 
108723                                        綠豆 豬展 蓮藕 鹽 

[108724 rows x 1 columns]


In [34]:
test=(x['Ingredients'][0:51])
print(test)
test.to_csv("database_back_before2.csv",header=True,index=False,encoding='utf-8-sig')

0                            五花肉 蛋 薯粉 南乳/紅腐乳 鹽巴 五香粉 胡椒粉
1     雞胸肉4大塊 (醃漬醬料)醬油膏 蜂蜜 鹽 黑胡椒 薑末 蒜末 韓式辣椒醬 醬油 蜂蜜 醋 ...
2                      馬鈴薯 蒜頭 奶油 水 牛奶 鹽 義式香料 白胡椒 莫札瑞拉起司
3                              義大利肉醬 乳酪絲 燙熟花椰菜 馬鈴薯 牛奶 鹽
4                馬鈴薯 培根 洋蔥 黑胡椒粉 鹽 鮮奶油 乳酪絲 奶油 義大利香料(可省略)
5     蝦子 小卷 小黃瓜 小蕃茄 紫洋蔥 牛奶洋蔥 衛生冰塊 薑片 檸檬汁 檸檬皮屑 香菜 大根的...
6                                     豬肝 牛奶 蔥 薑 鹽 冰塊 米酒
7                             白蘿蔔 工研糯米醋(白醋) 過濾水(開水) 糖 鹽
8                                      小黃瓜 醬油 糯米醋 米酒 砂糖
9         洋蔥約一顆 (浸泡用)鹽巴 (浸泡用)開水 (醃料)砂糖 (醃料)高粱醋或白醋 (醃料)水
10                        豬五花肉 八角 水 蒜苗 蒜頭 油 醬油 水 胡椒粉 冰糖
11                         肉燥 花瓜 蒜頭 調味料 醬油 糖 鹽巴 花瓜汁 太白粉
12                   黑豚大骨高湯 五花肉 紅蔥頭 醬油 冰糖 胡椒粉、五香粉 油蔥酥 鹽
13    豬肉(任何部位皆可) 蔥 薑片 料酒 雞蛋 豆干 【調味料】黃酒 【調味料】醬油 【調味料】糖 水
14         麻油 植物油 蒜頭 薑片 八角 辣豆瓣醬 辣椒醬(特辣) 番茄(切片) 番茄醬 醬油 水
15                 馬鈴薯 橄欖油 鹽 辣椒粉 胡椒粉/義式香料粉 Zyliss四合一刨刀器
16                 雞腿肉切塊 紅蘿蔔 (滾刀切塊) 馬鈴薯 (滾刀切塊) 咖哩塊 洋蔥切絲
17    麵疙瘩材料~中筋麵粉 太白粉 水 鹽 熱狗 花椰菜燙熟 紅蘿蔔塊燙熟 玉米筍燙熟 吃

In [ ]:
!pip install jieba

In [ ]:
!pip install -U ckiptagger[tfgpu,gdown]

In [11]:
import jieba
import jieba.posseg as pseg
def jieba_cut(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    strs=cut_word
    for str1 in strs:
        seg_list = jieba.cut(str1,use_paddle=True) # 使用paddle模式
        print("Paddle Mode: " + ' '.join(list(seg_list)))

    
    finsh=[]
    text = cut_word
    print(text)
    words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    for word, flag in words:
        if flag =="n" or flag== "nr" or flag=='x':
            print(word, flag)
            finsh.append(word)
        else:
            print(word, flag)
            #pass
    print(finsh)
    return finsh
jieba_cut(["麵疙瘩材料~中筋麵粉"])          

Paddle Mode: 麵疙瘩材料 ~ 中筋麵粉
['麵疙瘩材料~中筋麵粉']


AttributeError: 'list' object has no attribute 'decode'

In [23]:
def jieba_cut2(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    cut_list=cut_word
    finsh=''
    for str1 in cut_list:
        seg_list = jieba.lcut(str1, cut_all=False)
        print(seg_list)
        a=(' '.join(seg_list))
        print(a)
        print(finsh)
        finsh=finsh+a
        print(finsh)

    #print(finsh)
    return finsh
    
jieba_cut2(["麵疙瘩材料~中筋麵粉"]) 
    #words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    #for word, flag in words:
        #if flag =="n" or flag== "nr" or flag=='x':
            #print(word, flag)
            #finsh.append(word)
        #else:
            #print(word, flag)
            #pass
    

['麵疙瘩材料', '~', '中筋麵粉']
麵疙瘩材料 ~ 中筋麵粉

麵疙瘩材料 ~ 中筋麵粉


'麵疙瘩材料 ~ 中筋麵粉'

In [ ]:
import jieba
jieba.set_dictionary('./dict.txt')
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

In [ ]:
import jieba
import jieba.posseg as pseg
words = pseg.cut("我愛北京天安門") #jieba默认模式
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut("我愛北京天安門",use_paddle=True) #paddle模式
for word, flag in words:
    print('%s %s' % (word, flag))

In [ ]:
from __future__ import print_function, unicode_literals
import sys
sys.path.append("../")
import jieba
jieba.load_userdict("dict.txt")
import jieba.posseg as pseg

jieba.add_word('石墨烯')
jieba.add_word('凱特琳')
jieba.del_word('自定义词')

test_sent = (
"李小福是创新办主任也是云计算方面的专家; 什么是八一双鹿\n"
"例如我输入一个带“韩玉赏鉴”的标题，在自定义词库中也增加了此词为N类\n"
"「台中」正確應該不會被切開。mac上可分出「石墨烯」；此時又可以分出來凱特琳了。"
)
words = jieba.cut(test_sent)
print('/'.join(words))

print("="*40)

result = pseg.cut(test_sent)

for w in result:
    print(w.word, "/", w.flag, ", ", end=' ')

print("\n" + "="*40)

terms = jieba.cut('easy_install is great')
print('/'.join(terms))
terms = jieba.cut('python 的正则表达式是好用的')
print('/'.join(terms))

print("="*40)
# test frequency tune
testlist = [
('今天天气不错', ('今天', '天气')),
('如果放到post中将出错。', ('中', '将')),
('我们中出了一个叛徒', ('中', '出')),
]

for sent, seg in testlist:
    print('/'.join(jieba.cut(sent, HMM=False)))
    word = ''.join(seg)
    print('%s Before: %s, After: %s' % (word, jieba.get_FREQ(word), jieba.suggest_freq(seg, True)))
    print('/'.join(jieba.cut(sent, HMM=False)))
    print("-"*40)

  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314481 sha256=2762600f6817439c9789d9b8c817b376669303f7039cbc05bb213cedd365346f
  Stored in directory: c:\users\big data\appdata\local\pip\cache\wheels\24\aa\17\5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba


In [71]:
import pandas as pd

rowindex=['食材名稱']
df=pd.DataFrame(get_dict1,index=rowindex)
df=df.stack()
df.to_csv('database_clean4.csv',header=False,index=True,encoding='utf-8-sig')
y=pd.read_csv('database_clean4.csv',encoding='utf-8')
#print(y)
z=y.drop(columns='食材名稱',axis=1)
print(y.columns)
print(z)
z.to_csv('database_jieba_total_clean.csv',header=True,index=False,encoding='utf-8-sig')

Index(['食材名稱', '大根辣椒', '1'], dtype='object')
          大根辣椒      1
0       胡椒粉香料粉      1
1      麵疙瘩中筋麵粉      1
2         全張壽司      1
3         醃漬鹽巴      1
4         香料麵筋      1
...        ...    ...
56164       砂糖  15268
56165       醬油  17800
56166        糖  28301
56167        水  29408
56168        鹽  34663

[56169 rows x 2 columns]
